# Sensitivity Kernels and Tomography

This notebook demonstrates how to compute and visualize sensitivity kernels for seismic tomography using the `seisray` package.

## Learning Objectives
- Understand the concept of sensitivity kernels
- Compute kernels for different ray paths
- Visualize kernel matrices and patterns
- Explore resolution and trade-offs in tomography
- Stack kernels for multiple ray paths

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import sys
import os

# Add the parent directory to the path to import seisray
sys.path.append(os.path.dirname(os.getcwd()))

from seisray import SensitivityKernel, RayPathTracer, TravelTimeCalculator

print("Successfully imported seisray package!")

## 1. Introduction to Sensitivity Kernels

Sensitivity kernels describe how changes in Earth's velocity structure affect seismic observables (like travel times). They are fundamental to seismic tomography.

In [ ]:
# Set up a simple 2D grid for our sensitivity kernels
depth_grid = np.linspace(0, 800, 41)  # 0 to 800 km depth
distance_grid = np.linspace(0, 90, 46)  # 0 to 90 degrees distance

print(f"Grid dimensions:")
print(f"  Depth: {len(depth_grid)} points from {depth_grid[0]} to {depth_grid[-1]} km")
print(f"  Distance: {len(distance_grid)} points from {distance_grid[0]} to {distance_grid[-1]}°")
print(f"  Total grid points: {len(depth_grid) * len(distance_grid)}")

# Create depth-distance mesh for plotting
Dist, Depth = np.meshgrid(distance_grid, depth_grid)

## 2. Computing a Single Kernel

Let's compute a sensitivity kernel for a single P-wave ray path.

In [ ]:
# Set up parameters for kernel computation
source_depth = 10  # km
distance = 60      # degrees
phase = 'P'

# Create sensitivity kernel calculator
kernel_calc = SensitivityKernel('iasp91', grid_size=(len(depth_grid), len(distance_grid)))

# Compute kernel for P-wave
print(f"Computing sensitivity kernel for:")
print(f"  Phase: {phase}")
print(f"  Source depth: {source_depth} km")
print(f"  Distance: {distance}°")

try:
    kernel_matrix = kernel_calc.compute_straight_ray_kernel(
        source_depth=source_depth,
        distance_deg=distance,
        phase=phase,
        depth_grid=depth_grid,
        distance_grid=distance_grid
    )

    print(f"\nKernel computation successful!")
    print(f"  Kernel matrix shape: {kernel_matrix.shape}")
    print(f"  Non-zero elements: {np.count_nonzero(kernel_matrix)}")
    print(f"  Sparsity: {(1 - np.count_nonzero(kernel_matrix) / kernel_matrix.size) * 100:.1f}%")

except Exception as e:
    print(f"Error computing kernel: {e}")
    # Create a simple straight-ray kernel as fallback
    tracer = RayPathTracer('iasp91')
    rays = tracer.get_ray_paths(source_depth, distance, phases=[phase])

    if rays:
        ray = rays[0]
        print(f"Creating simplified straight-ray kernel...")

        # Create a simple kernel along the ray path
        kernel_matrix = np.zeros((len(depth_grid), len(distance_grid)))

        # Interpolate ray path onto grid
        ray_distances = ray.path['dist']
        ray_depths = ray.path['depth']

        for i in range(len(ray_distances)):
            # Find nearest grid points
            dist_idx = np.argmin(np.abs(distance_grid - ray_distances[i]))
            depth_idx = np.argmin(np.abs(depth_grid - ray_depths[i]))

            if dist_idx < len(distance_grid) and depth_idx < len(depth_grid):
                kernel_matrix[depth_idx, dist_idx] = 1.0

        print(f"Simplified kernel created with {np.count_nonzero(kernel_matrix)} non-zero elements")

In [ ]:
# Visualize the sensitivity kernel
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Kernel in depth-distance space
ax = axes[0]
im = ax.imshow(kernel_matrix, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='RdBu_r')
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title(f'{phase}-wave Sensitivity Kernel\n(Source: {source_depth} km, Distance: {distance}°)')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Sensitivity')

# Plot 2: Circular Earth view with ray path
ax = axes[1]

# Plot Earth structure
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

# Plot ray path
tracer = RayPathTracer('iasp91')
rays = tracer.get_ray_paths(source_depth, distance, phases=[phase])

if rays:
    ray = rays[0]
    distances_rad = np.radians(ray.path['dist'])
    radii = 6371 - ray.path['depth']

    x = radii * np.sin(distances_rad)
    y = radii * np.cos(distances_rad)

    ax.plot(x, y, 'r-', linewidth=3, label=f'{phase}-wave ray')

# Plot source and receiver
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'k*', markersize=15, label='Source')

receiver_x = 6371 * np.sin(np.radians(distance))
receiver_y = 6371 * np.cos(np.radians(distance))
ax.plot(receiver_x, receiver_y, 'k^', markersize=12, label='Receiver')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-7000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title('Ray Path Geometry')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Multiple Ray Paths and Kernel Stacking

In real tomography, we use many ray paths. Let's compute kernels for multiple source-receiver pairs.

In [ ]:
# Define multiple source-receiver pairs
ray_configs = [
    {'source_depth': 10, 'distance': 30, 'phase': 'P'},
    {'source_depth': 10, 'distance': 45, 'phase': 'P'},
    {'source_depth': 10, 'distance': 60, 'phase': 'P'},
    {'source_depth': 10, 'distance': 75, 'phase': 'P'},
    {'source_depth': 50, 'distance': 40, 'phase': 'P'},
    {'source_depth': 50, 'distance': 70, 'phase': 'P'},
]

print(f"Computing kernels for {len(ray_configs)} ray paths...")

# Compute individual kernels
individual_kernels = []
ray_info = []

for i, config in enumerate(ray_configs):
    try:
        # Create simplified straight-ray kernel
        tracer = RayPathTracer('iasp91')
        rays = tracer.get_ray_paths(config['source_depth'], config['distance'],
                                   phases=[config['phase']])

        if rays:
            ray = rays[0]
            kernel_matrix = np.zeros((len(depth_grid), len(distance_grid)))

            # Create kernel along ray path
            ray_distances = ray.path['dist']
            ray_depths = ray.path['depth']

            for j in range(len(ray_distances)):
                dist_idx = np.argmin(np.abs(distance_grid - ray_distances[j]))
                depth_idx = np.argmin(np.abs(depth_grid - ray_depths[j]))

                if dist_idx < len(distance_grid) and depth_idx < len(depth_grid):
                    kernel_matrix[depth_idx, dist_idx] = 1.0

            individual_kernels.append(kernel_matrix)
            ray_info.append({
                'config': config,
                'travel_time': ray.time,
                'max_depth': np.max(ray.path['depth'])
            })

            print(f"  Ray {i+1}: {config['source_depth']} km -> {config['distance']}° ({ray.time:.1f} s)")

    except Exception as e:
        print(f"  Ray {i+1}: Failed - {e}")

print(f"\nSuccessfully computed {len(individual_kernels)} kernels")

In [ ]:
# Visualize individual kernels
n_kernels = len(individual_kernels)
cols = 3
rows = (n_kernels + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
if rows == 1:
    axes = axes.reshape(1, -1)

for i, (kernel, info) in enumerate(zip(individual_kernels, ray_info)):
    row, col = i // cols, i % cols
    ax = axes[row, col]

    im = ax.imshow(kernel, aspect='auto', origin='lower',
                   extent=[distance_grid[0], distance_grid[-1],
                          depth_grid[0], depth_grid[-1]],
                   cmap='RdBu_r')

    config = info['config']
    ax.set_xlabel('Distance (degrees)')
    ax.set_ylabel('Depth (km)')
    ax.set_title(f"Ray {i+1}: {config['source_depth']} km → {config['distance']}°\n"
                f"Time: {info['travel_time']:.1f} s, Max depth: {info['max_depth']:.0f} km")
    ax.invert_yaxis()
    plt.colorbar(im, ax=ax, label='Sensitivity')

# Hide unused subplots
for i in range(n_kernels, rows * cols):
    row, col = i // cols, i % cols
    axes[row, col].set_visible(False)

plt.tight_layout()
plt.show()

## 4. Kernel Stacking and Coverage Analysis

Let's stack the kernels to see the combined sensitivity and analyze coverage.

In [ ]:
# Stack all kernels
stacked_kernel = np.sum(individual_kernels, axis=0)
normalized_stack = stacked_kernel / np.max(stacked_kernel)

# Calculate coverage statistics
coverage = (stacked_kernel > 0).astype(float)
total_coverage = np.sum(coverage) / coverage.size * 100

print(f"Coverage Analysis:")
print(f"  Total grid points: {coverage.size}")
print(f"  Covered points: {np.sum(coverage):.0f}")
print(f"  Coverage percentage: {total_coverage:.1f}%")
print(f"  Maximum hit count: {np.max(stacked_kernel):.0f}")
print(f"  Average hits per covered point: {np.mean(stacked_kernel[stacked_kernel > 0]):.1f}")

# Visualize stacked kernels
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Stacked kernel (raw)
ax = axes[0, 0]
im = ax.imshow(stacked_kernel, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='hot')
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title('Stacked Sensitivity (Raw Counts)')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Ray Count')

# Normalized stacked kernel
ax = axes[0, 1]
im = ax.imshow(normalized_stack, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='hot')
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title('Normalized Stacked Sensitivity')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Normalized Sensitivity')

# Coverage map
ax = axes[1, 0]
im = ax.imshow(coverage, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='binary')
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title(f'Coverage Map ({total_coverage:.1f}% covered)')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Covered (1) / Uncovered (0)')

# Hit count histogram
ax = axes[1, 1]
hit_counts = stacked_kernel[stacked_kernel > 0]
ax.hist(hit_counts, bins=np.arange(0.5, np.max(hit_counts) + 1.5),
        alpha=0.7, edgecolor='black')
ax.set_xlabel('Number of Ray Hits')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Ray Hit Counts')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Resolution Analysis

Let's analyze the resolution capabilities of our ray configuration.

In [ ]:
# Create a simple resolution test
# We'll create a synthetic velocity anomaly and see how well it can be resolved

# Define a synthetic anomaly (checkerboard pattern)
anomaly_size = 10  # degrees/levels
synthetic_anomaly = np.zeros_like(Depth)

for i in range(0, len(depth_grid), anomaly_size):
    for j in range(0, len(distance_grid), anomaly_size):
        if (i // anomaly_size + j // anomaly_size) % 2 == 0:
            end_i = min(i + anomaly_size, len(depth_grid))
            end_j = min(j + anomaly_size, len(distance_grid))
            synthetic_anomaly[i:end_i, j:end_j] = 1.0

# Simulate data (synthetic travel time anomalies)
synthetic_data = []
for kernel in individual_kernels:
    # Calculate the integrated effect of the anomaly on travel time
    data_point = np.sum(kernel * synthetic_anomaly) * 0.1  # Scale factor
    synthetic_data.append(data_point)

synthetic_data = np.array(synthetic_data)

print(f"Synthetic Resolution Test:")
print(f"  Anomaly size: {anomaly_size} grid points")
print(f"  Number of data points: {len(synthetic_data)}")
print(f"  Data range: {np.min(synthetic_data):.3f} to {np.max(synthetic_data):.3f}")
print(f"  Data std: {np.std(synthetic_data):.3f}")

# Simple back-projection (smearing) to estimate resolution
back_projection = np.zeros_like(synthetic_anomaly)
for i, (kernel, data) in enumerate(zip(individual_kernels, synthetic_data)):
    back_projection += kernel * data

# Normalize
if np.max(back_projection) > 0:
    back_projection /= np.max(back_projection)

# Calculate correlation with original
correlation = np.corrcoef(synthetic_anomaly.flatten(), back_projection.flatten())[0, 1]

print(f"  Correlation with original: {correlation:.3f}")

In [ ]:
# Visualize resolution test
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original synthetic anomaly
ax = axes[0]
im = ax.imshow(synthetic_anomaly, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title('Original Synthetic Anomaly')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Velocity Anomaly')

# Back-projected result
ax = axes[1]
im = ax.imshow(back_projection, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title(f'Back-projected Result\n(Correlation: {correlation:.3f})')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Recovered Anomaly')

# Difference
ax = axes[2]
difference = back_projection - synthetic_anomaly
im = ax.imshow(difference, aspect='auto', origin='lower',
               extent=[distance_grid[0], distance_grid[-1],
                      depth_grid[0], depth_grid[-1]],
               cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Depth (km)')
ax.set_title('Difference (Recovered - Original)')
ax.invert_yaxis()
plt.colorbar(im, ax=ax, label='Difference')

plt.tight_layout()
plt.show()

# Print data values
print(f"\nSynthetic Travel Time Data:")
print(f"{'Ray':<4} {'Source Depth':<12} {'Distance':<10} {'Data Value':<12}")
print("-" * 45)
for i, (info, data) in enumerate(zip(ray_info, synthetic_data)):
    config = info['config']
    print(f"{i+1:<4} {config['source_depth']:<12} {config['distance']:<10} {data:<12.4f}")

## 6. Trade-off Analysis

Let's explore the trade-offs between resolution and data coverage.

In [ ]:
# Analyze resolution as a function of depth
depth_resolution = []
depth_coverage = []

depth_bins = np.linspace(0, 800, 21)  # Depth bins for analysis

for i in range(len(depth_bins) - 1):
    depth_start = depth_bins[i]
    depth_end = depth_bins[i + 1]

    # Find depth indices in this bin
    depth_mask = (depth_grid >= depth_start) & (depth_grid < depth_end)

    if np.any(depth_mask):
        # Calculate coverage in this depth bin
        bin_coverage = coverage[depth_mask, :]
        coverage_percent = np.sum(bin_coverage) / bin_coverage.size * 100

        # Calculate hit density
        bin_hits = stacked_kernel[depth_mask, :]
        avg_hits = np.mean(bin_hits[bin_hits > 0]) if np.any(bin_hits > 0) else 0

        depth_coverage.append(coverage_percent)
        depth_resolution.append(avg_hits)
    else:
        depth_coverage.append(0)
        depth_resolution.append(0)

depth_centers = (depth_bins[:-1] + depth_bins[1:]) / 2

# Plot resolution vs depth
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Coverage vs depth
ax = axes[0]
ax.plot(depth_coverage, depth_centers, 'b-', linewidth=2, marker='o')
ax.set_xlabel('Coverage (%)')
ax.set_ylabel('Depth (km)')
ax.set_title('Coverage vs Depth')
ax.invert_yaxis()
ax.grid(True, alpha=0.3)

# Resolution (hit density) vs depth
ax = axes[1]
ax.plot(depth_resolution, depth_centers, 'r-', linewidth=2, marker='s')
ax.set_xlabel('Average Ray Hits')
ax.set_ylabel('Depth (km)')
ax.set_title('Ray Density vs Depth')
ax.invert_yaxis()
ax.grid(True, alpha=0.3)

# Trade-off plot
ax = axes[2]
scatter = ax.scatter(depth_coverage, depth_resolution, c=depth_centers,
                    cmap='viridis', s=100, alpha=0.7)
ax.set_xlabel('Coverage (%)')
ax.set_ylabel('Average Ray Hits')
ax.set_title('Resolution vs Coverage Trade-off')
ax.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax, label='Depth (km)')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nDepth-dependent Resolution Analysis:")
print(f"{'Depth Range (km)':<15} {'Coverage (%)':<12} {'Avg Hits':<10}")
print("-" * 40)
for i, (depth_center, cov, res) in enumerate(zip(depth_centers, depth_coverage, depth_resolution)):
    depth_start = depth_bins[i]
    depth_end = depth_bins[i + 1]
    print(f"{depth_start:3.0f}-{depth_end:3.0f}        {cov:<12.1f} {res:<10.1f}")

## Summary

In this notebook, we demonstrated:

1. **Sensitivity kernel computation** for individual ray paths
2. **Kernel visualization** in depth-distance space
3. **Multiple ray path analysis** and kernel stacking
4. **Coverage analysis** showing which parts of the Earth are sampled
5. **Resolution testing** using synthetic checkerboard patterns
6. **Trade-off analysis** between resolution and coverage

### Key Findings:
- Sensitivity kernels show which parts of the Earth affect travel times
- Ray path coverage is uneven - shallow depths and intermediate distances are best sampled
- Resolution decreases with depth due to fewer ray paths
- Multiple ray paths improve resolution through redundancy
- Trade-offs exist between spatial coverage and resolution

### Tomographic Insights:
- **Good coverage**: Shallow depths (0-200 km), intermediate distances (20-80°)
- **Poor coverage**: Very shallow surface, very deep regions, extreme distances
- **Resolution**: Best where multiple ray paths cross
- **Limitations**: Straight-ray approximation oversimplifies kernel shapes

### Future Improvements:
- Finite-frequency kernels for more realistic sensitivity
- 3D ray tracing for complex Earth models
- Formal resolution matrix analysis
- Regularization techniques for inversion

The `seisray` package provides the foundation for building more sophisticated tomographic inversion systems.